# Dynamic Programming
3 steps for analyzing a DP problem:  
1. Analyze overlapping subproblems
2. Store the result of subproblems
3. Construct solution

## Example: Rob cutting

In [1]:
from collections import defaultdict

original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30]
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i+1] = p

### brute-force recursion

In [2]:
solution = {} # store optimal cutting point

In [3]:
from functools import lru_cache

In [4]:
# if n not in price list, price[n] = 0, must split
# time complexity: O(2^n)
# @lru_cache(maxsize=2**10)
def r(n):
    max_price, split_point =  max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)],
        key = lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    
    return max_price

In [5]:
# r(143)

In [6]:
solution

{}

Side notes:  
Python 面向函数的编程
1. 函数可以作为argument传参
2. 可以赋值给变量
3. 可以作为return

### decorator
```python
@decorator
def func():
    ...
```
is equivalent to 
```python
func = decorator(func)
```
Advantage: 
* don't need to modify the original function
* avoid repetitive work

not recommended way

In [7]:
def func_1(n):
    for i in range(n):
        print(n)

In [8]:
def call_time(func_1, arg): # 脚手架程序 scaffold (programming)
    start = time.time()
    func_1(arg)
    print('used time: {}'.format(time.time() - start))

recommended way, using decorator

In [9]:
from functools import wraps

In [10]:
function_called_time = defaultdict(int)

def get_call_time(func):
    @wraps(func)
    def _inner(arg): ## *args, **kwargs
        """It's inner function"""
        global function_called_time
        function_called_time[func.__name__] += 1
        result = func(arg)
        print('function called time is : {}'.format(function_called_time[func.__name__]))
        return result
    return _inner

In [11]:
@get_call_time
def func_1(n):
    """
    @param n: is the number of customers
    @return int: the customers value point
    """
    for i in range(n):
        print(n)
    return 0

In [12]:
help(func_1)

Help on function func_1 in module __main__:

func_1(n)
    @param n: is the number of customers
    @return int: the customers value point



In [13]:
func_1(5)

5
5
5
5
5
function called time is : 1


0

### Recursion with memoization

In [14]:
def memo(func):
    cache = {}
    @wraps(func)
    def _wrap(n): # *args, **kwargs
        if n in cache:
            res = cache[n]
        else:
            res = func(n)
            cache[n] = res
        return res
    return _wrap

In [15]:
@memo
def r(n):
    max_price, split_point = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (split_point, n - split_point)
    
    return max_price

In [16]:
r(231)

691

#### Parse solution

In [17]:
def not_cut(split): return split == 0

def parse_solution(target_length, revenue_solution):
    # get the lift and right split point
    left, right = revenue_solution[target_length] 
    
    if not_cut(left): return [right] # if no further cut, return
    
    return parse_solution(left, revenue_solution) + parse_solution(right, revenue_solution)

In [18]:
parse_solution(19, solution)

[3, 6, 10]

## [Edit Distance](https://en.wikipedia.org/wiki/Edit_distance)
Measuring how dissimilar two strings (DNA sequence) are by counting the number of operations required to transform one string into the other.  
#### Edit operations ( => edit distance): 
* insertion => 1
* deletion => 1
* substitution => 2

#### Aplication scenarios:
* spell correction
* evaluating machine translation and speech recognition
* find duplicates

In [19]:
solution = defaultdict(str)

In [20]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if not string1: 
        if string2: solution[(string1, string2)] = 'ADD {}'.format(string2) # edited
        return len(string2)
    if not string2: 
        if string1: solution[(string1, string2)] = 'DEL {}'.format(string1) # edited
        return len(string1)
    
    tail_s1, tail_s2 = string1[-1], string2[-1]
    
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)), # string1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)), # string1 add tail of string2  
    ]
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]), '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))
    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    # save the subproblem solution
    solution[(string1, string2)] = operation
    
    return min_distance

In [21]:
def parse_solution(string1, string2, ed_sol, ops):
    
    op = ed_sol[(string1, string2)]
    
    # exit condition
    if not string1 or not string2:
        if op: 
            ops.append(((string1, string2), op))
        return ops
    
    ops.append(((string1, string2), op))
    if op.startswith('SUB') or op == '':
        return parse_solution(string1[:-1], string2[:-1], ed_sol, ops)
    if op.startswith('ADD'):
        return parse_solution(string1, string2[:-1], ed_sol, ops)
    if op.startswith('DEL'):
        return parse_solution(string1[:-1], string2, ed_sol, ops)

In [22]:
edit_distance('berkeley','berklee')

2

In [23]:
parse_solution('berkeley','berklee', solution, [])

[(('berkeley', 'berklee'), 'SUB y => e'),
 (('berkele', 'berkle'), ''),
 (('berkel', 'berkl'), ''),
 (('berke', 'berk'), 'DEL e'),
 (('berk', 'berk'), ''),
 (('ber', 'ber'), ''),
 (('be', 'be'), ''),
 (('b', 'b'), '')]

In [24]:
edit_distance('alabama', 'blaska')

4

In [25]:
parse_solution('alabama', 'blaska', solution, [])

[(('alabama', 'blaska'), ''),
 (('alabam', 'blask'), 'DEL m'),
 (('alaba', 'blask'), 'SUB a => k'),
 (('alab', 'blas'), 'SUB b => s'),
 (('ala', 'bla'), ''),
 (('al', 'bl'), ''),
 (('a', 'b'), 'SUB a => b')]